# Tokenization

In [1]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt -O input.txt

with open('input.txt', 'r', encoding='utf8') as f:
    text = f.read()

--2024-09-27 20:50:26--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.82MB/s    in 0.6s    

2024-09-27 20:50:27 (1.82 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
from tokenizer import Tokenizer

tokenizer = Tokenizer(text, vocab_size=300)

tokenizer.decode(tokenizer.encode("Hello, how are you?"))

'Hello, how are you?'